In [1]:
USE_TF = False
USE_DISTILBERT = True
DP_MODEL_PATH = './models/bert_base_rus_joined_sent_cased'
DP_CONFIG_PATH = './models/bert_base_rus_joined_sent_cased/dp_config.json'
PT_BERT_CONFIG_PATH = './models/bert_base_rus_joined_sent_cased_pt/config.json'
PT_BERT_MODEL_PATH = './models/bert_base_rus_joined_sent_cased_pt/'
PT_DB_CONFIG_PATH = './models/distilbert_rus_joined_sent_cased_pt/config.json'
PT_DB_MODEL_PATH = './models/distilbert_rus_joined_sent_cased_pt/'

In [2]:
# compability
def dp_model_wrapper(self, *args):
    new_preds = list()
    preds = self.__call(*args)
    for n_pred in range(len(preds[0])):
        new_preds.append({
            'label': preds[0][n_pred],
            'score': max(preds[1][n_pred]),
        })
    return new_preds

In [3]:
import os

if USE_TF:
    from deeppavlov.core.common.file import read_json
    from deeppavlov import build_model
    from deeppavlov import Chainer
    # Override standart call to identical output
    Chainer.__call = Chainer.__call__
    Chainer.__call__ = dp_model_wrapper
    
    config = read_json(DP_CONFIG_PATH)
    config['metadata']['variables']['MODEL_PATH'] = os.path.abspath(DP_MODEL_PATH)
    model = build_model(config)
elif USE_DISTILBERT:
    import torch
    from transformers import DistilBertTokenizer, DistilBertConfig, DistilBertForSequenceClassification, TextClassificationPipeline
    import json
    config = DistilBertConfig.from_pretrained(PT_DB_CONFIG_PATH)
    classes_dict = json.load(open(os.path.join(PT_DB_MODEL_PATH, 'classes.dict')))
    config.label2id = classes_dict['label2id']
    config.id2label = {i[1]:i[0] for i in classes_dict['label2id'].items()}
    tokenizer = DistilBertTokenizer.from_pretrained(
        PT_DB_MODEL_PATH,
        do_lower_case=False,
    )
    distilbert_model = DistilBertForSequenceClassification.from_pretrained(
        PT_DB_MODEL_PATH, 
        from_tf=False, 
        config=config, 
    )
    distilbert_model.load_state_dict(torch.load(os.path.join(PT_DB_MODEL_PATH, 'pytorch_model.bin')))
    
    device = 0 if torch.cuda.is_available() else -1
    
    model = TextClassificationPipeline(model=distilbert_model, tokenizer=tokenizer, device=device)
else:
    import torch
    from transformers import BertTokenizer, BertConfig, BertForSequenceClassification, TextClassificationPipeline
    import json
    config = BertConfig.from_pretrained(PT_BERT_CONFIG_PATH)
    classes_dict = json.load(open(os.path.join(PT_BERT_MODEL_PATH, 'classes.dict')))
    config.label2id = classes_dict['label2id']
    config.id2label = {i[1]:i[0] for i in classes_dict['label2id'].items()}
    tokenizer = BertTokenizer.from_pretrained(
        PT_BERT_MODEL_PATH,
        do_lower_case=False,
    )
    bert_model = BertForSequenceClassification.from_pretrained(
        PT_BERT_MODEL_PATH, 
        from_tf=False, 
        config=config, 
    )
    bert_model.load_state_dict(torch.load(os.path.join(PT_BERT_MODEL_PATH, 'pytorch_model.bin')))
    
    device = 0 if torch.cuda.is_available() else -1
    
    model = TextClassificationPipeline(model=bert_model, tokenizer=tokenizer, device=device)

In [4]:
model([
    'Скупой платит дважды. Пойду работать к скупому.',
    'Такси везет меня на работу. Раздумываю приплатить, чтобы меня втащили на пятый этаж. Лифта то нет :(',
    'В настройках телефона пробовали ставить приоритет "Только 3G"? Всегда такая сеть там была? #билайн',
    'Для всех родных &lt;3 Искренне вас люблю , родные мои :) Скучаю по вам',
    'Как же я соскучилась по Никите((( вновь хочу его увидеть!',
    'с добреньким утречком и последними днями моих каникул:(',
])

[{'label': 'humor', 'score': 0.9755868911743164},
 {'label': 'negative', 'score': 0.9982805848121643},
 {'label': 'neutral', 'score': 0.9273685812950134},
 {'label': 'positive', 'score': 0.9901606440544128},
 {'label': 'negative', 'score': 0.9865018725395203},
 {'label': 'speech', 'score': 0.5939101576805115}]